# Supercharge Your Jupyter Notebook: SQL Command Magic for IPython

Jupyter Notebooks are a powerful tool for data analysis, but working with databases inside them has often been cumbersome. Wouldn’t it be great if you could seamlessly run SQL queries against Microsoft SQL Server inside your notebook—without switching environments? 

Meet **SQL Command Magic for IPython**, an IPython extension that integrates **sqlcmd** directly into your Jupyter workflow. With this extension, you can execute SQL commands effortlessly, connect to databases dynamically, and even run SQL scripts—all from your notebook.

## The Problem: Setting Up a Sample Database

I was working on a tutorial notebook that required a **real database** to demonstrate SQL queries. However, setting up a Microsoft SQL Server instance with test data inside Jupyter turned out to be a hassle. Installing drivers, configuring connections, and loading data manually was tedious. 

That’s when I decided to **build a solution**: SQL Command Magic.

Now, with a few commands, you can:
✅ Connect to SQL Server effortlessly.  
✅ Execute SQL queries directly in Jupyter/IPython.  
✅ Run SQL commands in batches using `GO` statements.  
✅ Execute external SQL scripts seamlessly.  
✅ Substitute Python variables inside SQL queries.  
✅ Enable debugging to analyze execution details.  

## Getting Started

Let's explore how to use the ipython extension.

**Installation**:  
Install SQL Command Magic via pip:

In [ ]:
pip install sqlcmd-magic

Ensure that `sqlcmd` is installed and accessible in your system’s PATH. If you haven’t installed it, check out Microsoft’s [official sqlcmd documentation](https://docs.microsoft.com/en-us/sql/tools/sqlcmd-utility) to get started.


**Loading the Extension**:  
To enable the extension in an IPython or Jupyter Notebook session, simply run:


In [ ]:
%load_ext sqlcmd

The extension is now loaded to your environment and ready to use.

## Connecting to the Northwind Database

Instead of setting up a new database manually, let’s work with **Northwind**, a classic SQL Server sample database. First, we need to connect to the server. Fil out the username with the username for SQL authentication to your DB and add the `SSMS_PASSWORD` to the .env file in this directory:

In [ ]:
import os
password = os.getenv("SSMS_PASSWORD")
username = "sa"

Now, let's connect:

In [ ]:
%sqlcmd master --server=localhost --username={username} --password={password}

Connection string set: mssql+sqlcmd:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3Dlocalhost%3BDATABASE%3Dmaster%3BUID%3Dsa%3BPWD%3Dmypassword1234%21%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dyes%3B
Parsed connection info: {'server': 'localhost', 'database': 'master', 'username': 'sa', 'password': '***'}


Using the cell magic, we can create the database:

In [ ]:
%%sqlcmd
IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'Northwind')
BEGIN
    CREATE DATABASE Northwind;
END;

Let's check wether we were successfull:

In [ ]:
%%sqlcmd 
SELECT name, database_id, create_date FROM sys.databases WHERE name = 'Northwind';
GO
USE Northwind;
GO

,name,database_id,create_date
0,Northwind,5,2024-11-15 14:17:07.740



(1 rows affected)
Changed database context to 'Northwind'.



## Downloading and the logical backup and creating the DB

The microsof server samples github holds an complete logical backup script of the database. We can run the script on the server to completely initializa and fill the database. First, let's download the logical backup:

In [ ]:
!curl -o instnwnd.sql https://raw.githubusercontent.com/microsoft/sql-server-samples/master/samples/databases/northwind-pubs/instnwnd.sql

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1025k  100 1025k    0     0  1847k      0 --:--:-- --:--:-- --:--:-- 1853k


We can now execute the script using the `EXECUTE_SQL_FILE` command. The command is not a SQL command, but part of the ipython-sqlcmd python package. It can only be executed on the top scope of any SQL cell, that is outside of `BEGIN ... END` blocks:

In [ ]:
%%sqlcmd
EXECUTE_SQL_FILE 'instnwnd.sql'

output
------
NULL
(1 rows affected)
executing script at c:\Users\acisse\OneDrive - bridgingIT-Gruppe\003_CodeWorkspace\001_cookbooks\azure-cookbook\011-data-factory-python-sdks\instnwnd.sql




## Running Queries on Northwind

Now that we have a working **Northwind** database, let’s explore its data. After running, we can check wether the tables have been initiated correctly:

In [ ]:
%%sqlcmd 
SELECT TOP 5 * FROM Customers;

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67



(5 rows affected)


Now, let's query the Products Table

In [ ]:
%%sqlcmd 
SELECT TOP 5 * FROM Products;

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.0000,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.0000,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0000,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.0000,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.3500,0,0,0,1



(5 rows affected)


### Debug Mode for Troubleshooting
Want to see exactly what’s happening under the hood? Use the `--debug` flag.

In [ ]:
%%sqlcmd --debug
SELECT TOP 5 * FROM Products;

This helps diagnose issues in real time. You can CTRL + Click the file path to inspect the executed file.

## Why Use SQL Command Magic?

- **Seamless SQL Execution:**  
Write SQL inside notebooks without external scripts.
- **Reproducibility:**  
Keep SQL queries embedded with analysis code.
- **Enhanced Debugging:**  
Debugging tools make troubleshooting easy.
- **Integration:**  
Works smoothly with Pandas and other Python data tools.

## Conclusion

SQL Command Magic for IPython bridges the gap between Jupyter and SQL Server. Whether you are analyzing data, automating database tasks, or teaching SQL, this tool makes it effortless. 